# Acoustical statistics

This notebook generates panels for Figure 1 comparing the acoustical environments in the CR and PR conditions.
You need to have downloaded the `zebf-social-acoustical-stats.zip` file from the [Acoustical Statistics of Zebra Finch Breeding](https://doi.org/10.6084/m9.figshare.27961518) dataset and unpacked it in the `datasets` folder.

In [ ]:
import os
import sys

sys.path.insert(0, "../scripts")
# disable locking - texture summary stats may be on an NFS share
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

In [ ]:
# graphics parameters
import matplotlib as mpl

tickparams = {
    "major.size": 2,
    "minor.size": 1,
    "labelsize": "small",
    "direction": "out",
}
grparams = {
    "font": {"size": 6},
    "axes": {"linewidth": 0.5, "unicode_minus": False, "titlepad": 2.0},
    "lines": {"linewidth": 1.0},
    "xtick": tickparams,
    "ytick": tickparams,
    "image": {"aspect": "auto", "origin": "lower"},
    "pdf": {"fonttype": 42},
}
for k, v in grparams.items():
    mpl.rc(k, **v)

corr_cmap = "seismic"
condition_colors = ["#F8766D", "#00BFC4", "#000000"]

In [ ]:
import datetime
from pathlib import Path

import ewave
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from filters import AWeightTransform, SpectrogramTransform

In [ ]:
def read_summary_stats(path):
    summary = pd.read_csv(path)
    summary["date"] = pd.to_datetime(summary.date)
    summary["time"] = pd.to_timedelta(summary.time)
    summary["timestamp"] = summary.date + summary.time
    summary["recording_day"] = (summary.date - summary.date[0]).dt.days
    summary["hour"] = summary.time.dt.total_seconds() / 3600
    summary["is_daytime"] = (summary.hour > 6) & (summary.hour < 22)
    return summary

In [ ]:
def plot_summary(axes, df):
    axes.plot(df.timestamp, df.ampl_q50, ".", markersize=2)
    axes.vlines(df.timestamp, df.ampl_q25, df.ampl_q75, alpha=0.1, linewidths=1)
    axes.plot(df.timestamp, df.ampl_q100, "x", markersize=2, alpha=0.1)
    axes.set_ylabel("Amplitude (dBA SPL)")
    axes.set_xlabel("Time")
    axes.set_ylim(10, 110)

def plot_amplitudes(axes, log_spec, amplitude_envelope, fgrid, tgrid):
    pos = axes[0].imshow(log_spec, aspect="auto", origin="lower", vmin=0, vmax=80, 
                   interpolation="none",
                   extent=(tgrid[0], tgrid[-1], fgrid[0] / 1000, fgrid[-1] / 1000))
    axes[0].set_ylabel("Freq (kHz)")
    axes[1].plot(tgrid, amplitude_envelope, linewidth=0.5)
    axes[1].set_ylabel("Amplitude (dBA SPL)")
    axes[1].set_ylim(5, 90)
    axes[1].set_xlabel("Time (s)")
    return pos

In [ ]:
# dataset path
dataset = Path("../datasets/zebf-social-acoustical-stats/")
# spectrogram parameters
window_size = 0.010
max_frequency = 8000

## Amplitude statistics and example spectrograms

Amplitude statistics have been pre-calculated using the `scripts/amplitude-stats.py` script from long recording files that are too large to include in the dataset.

### Colony-reared

The colony nestbox recording started when the chicks were 26-29 days post hatch.

In [ ]:
cr_summary = read_summary_stats(dataset / "amplitude_cobalt-57_colony.csv")
# this is the example segment where there is song
segment_time = datetime.datetime(2024, 1, 26, 19, 40)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(3, 1.5), dpi=300)
plot_summary(axes, cr_summary)
idx_example = cr_summary.timestamp.searchsorted(segment_time)
axes.plot(cr_summary.timestamp.iloc[idx_example], cr_summary.ampl_q100.iloc[idx_example], 'x', markersize=2)
#fig.savefig("../figures/cr_cobalt_57_amplitude.pdf")

Example spectrograms. Note that this segment is *not* used in the texture analysis. I had to work pretty hard
to find the dad singing.

In [ ]:
# this is output by the amplitude-stats script
dBSPL_correction = 154401.9670647288
example_segment_file = dataset / "example_cobalt_57_family-colony-nestbox_20240126-203402.wav"
with ewave.open(example_segment_file, mode="r") as fp:
    example_segment = fp.read()
    sampling_rate = fp.sampling_rate

In [ ]:
raw_data = example_segment * dBSPL_correction
a_weighter = AWeightTransform(sampling_rate)
stfter = SpectrogramTransform(window_size, sampling_rate, max_frequency)
fgrid = stfter.freq
filtered_data = a_weighter.transform(raw_data)
spec = stfter.transform(filtered_data, scaling=None)
log_spec = 10 * np.log10(spec / stfter.scale1)
tgrid = stfter.tgrid(spec)
amplitude_envelope = 10 * np.log10(spec.sum(0) / stfter.scale2 * stfter.freq_res)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(2.5, 1.5), dpi=300)
pos = plot_amplitudes(axes, log_spec, amplitude_envelope, fgrid, tgrid)
axes[0].vlines([88, 93], ymin=fgrid[0] / 1000, ymax=fgrid[-1] / 1000, color="white", linewidths=0.5)
cax = fig.add_axes((0.95, 0.55, 0.01, 0.30))
cax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
cax.yaxis.tick_right()
cbar = fig.colorbar(pos, cax=cax, ticks=(0,40,80))
fig.savefig("../figures/cr_cobalt_57_example_5m.pdf")

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(1.75, 1.5), dpi=300)
plot_amplitudes(axes, log_spec, amplitude_envelope, fgrid, tgrid)
axes[1].set_xlim(88, 93)
fig.savefig("../figures/cr_cobalt_57_example_5s.pdf")

### Pair-reared

The PR nestbox recording started when the chicks were 4-11 dph. We want to compare to a comparable period
in the CR recording. Starting 23 days after start of recording gives us a starting age of 27-34 dph.

In [ ]:
pr_summary = read_summary_stats(dataset / "amplitude_purple-166_soundbox.csv")
# skip forward 23 days
start_time = pr_summary.date.iloc[0] + datetime.timedelta(days=23) + cr_summary.time.iloc[0]
end_time = start_time + (cr_summary.timestamp.iloc[-1] - cr_summary.timestamp.iloc[0])
pr_summary_recent = pr_summary.loc[(pr_summary.timestamp > start_time) &  (pr_summary.timestamp <= end_time)]
# this is the example segment where there is song
segment_time = datetime.datetime(2024, 3, 24, 15, 15)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(3, 1.5), dpi=300)
plot_summary(axes, pr_summary_recent)
idx_example = pr_summary_recent.timestamp.searchsorted(segment_time)
axes.plot(pr_summary_recent.timestamp.iloc[idx_example], pr_summary_recent.ampl_q100.iloc[idx_example], 'x', markersize=2)
fig.savefig("../figures/pr_purple_166_amplitude.pdf")

In [ ]:
# this is output by the amplitude-stats script
dBSPL_correction = 57925.56019476335
example_segment_file = dataset / "example_purple_166_family-pair-nestbox_20240324-154855.wav"
with ewave.open(example_segment_file, mode="r") as fp:
    example_segment = fp.read()
    sampling_rate = fp.sampling_rate

In [ ]:
raw_data = example_segment * dBSPL_correction
a_weighter = AWeightTransform(sampling_rate)
stfter = SpectrogramTransform(window_size, sampling_rate, max_frequency)
fgrid = stfter.freq
filtered_data = a_weighter.transform(raw_data)
spec = stfter.transform(filtered_data, scaling=None)
log_spec = 10 * np.log10(spec / stfter.scale1)
tgrid = stfter.tgrid(spec)
amplitude_envelope = 10 * np.log10(spec.sum(0) / stfter.scale2 * stfter.freq_res)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(2.5, 1.5), dpi=300)
plot_amplitudes(axes, log_spec, amplitude_envelope, fgrid, tgrid)
axes[0].vlines([54.3, 59.3], ymin=fgrid[0] / 1000, ymax=fgrid[-1] / 1000, color="white", linewidths=0.5)
#axes[1].vlines([417, 422], ymin=20, ymax=90, color="blue", linewidths=0.5)
axes[1].set_xlabel("Time (s)")
fig.savefig("../figures/pr_purple_166_example_5m.pdf")

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(1.75, 1.5), dpi=300)
plot_amplitudes(axes, log_spec, amplitude_envelope, fgrid, tgrid)
#axes[1].set_xlim(71.5, 76.5)
axes[1].set_xlim(54.3, 59.3)
fig.savefig("../figures/pr_purple_166_example_5s.pdf")

## Texture statistics

Before running the cells below, run the `texture-stats.py` script to analyze samples from the colony nestbox, clean recordings of song, and pink noise (we use the same sample from the McDermott source distribution):

``` shell
venv/bin/python scripts/texture-stats.py --output build/cr-family_texture-stats.h5 datasets/zebf-social-acoustical-stats/cobalt_57_family*wav
```

``` shell
venv/bin/python scripts/texture-stats.py --output build/clean-songs_texture-stats.h5 datasets/zebf-social-acoustical-stats/*songs*wav
```

``` shell
venv/bin/python scripts/texture-stats.py --duration 20 --output build/pink-noise_texture-stats.h5 datasets/zebf-social-acoustical-stats/pink_noise_20s_20kHz.wav
```


In [ ]:
def collect_stats(arf_file, name: str, axis: int):
    collected = [entry[name] for entry in arf_file.values() if isinstance(entry, h5py.Group) and name in entry]
    return np.stack(collected, axis=axis)

In [ ]:
song_stats = h5py.File("../build/clean-songs_texture-stats.h5", "r")
colony_stats = h5py.File("../build/cr-family_texture-stats.h5", "r")
noise_stats = h5py.File("../build/pink-noise_texture-stats.h5", "r")

In [ ]:
# pick out the labels for the graphs
erb_filter_locations = song_stats["erb_filters"].attrs["locations"]
erb_idx = np.arange(erb_filter_locations.size)
erb_pos = [0, 9, 19, 29]
erb_labels = [f"{label:.0f}" for label in erb_filter_locations[erb_pos]]
mps_filter_locations = song_stats["mps_filters"].attrs["locations"]
mps_pos = [4, 9, 14, 19]
mps_labels = [f"{label:.1f}" for label in mps_filter_locations[mps_pos]]
c12_filter_locations = song_stats["c12_filters"].attrs["locations"]
c2_pos = [0, 4]
c2_lbl = [f"{freq:.1f}" for freq in c12_filter_locations[c2_pos]]

## Comparing colony noise to song stats

More detailed plots are below this section

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, figsize=(2.5, 2), dpi=150)
for i, statfile in enumerate((colony_stats, song_stats)):
    env_mean = collect_stats(statfile, "envelope_means", axis=1)[1:-1]
    env_std = collect_stats(statfile, "envelope_stdvs", axis=1)[1:-1]
    env_skew = collect_stats(statfile, "envelope_skew", axis=1)[1:-1]
    env_kurt = collect_stats(statfile, "envelope_kurtosis", axis=1)[1:-1]
    axes[0,0].errorbar(erb_idx, env_mean.mean(1), yerr=env_mean.std(1) / np.sqrt(env_mean.shape[1]), color=condition_colors[i])
    axes[0,0].set_ylabel("Mean")
    axes[0,0].set_ylim(0, env_mean.max() * 1.1)
    axes[1,0].errorbar(erb_idx, env_std.mean(1), yerr=env_std.std(1) / np.sqrt(env_std.shape[1]), color=condition_colors[i])
    axes[1,0].set_ylabel("Std/Mean")
    axes[0,1].errorbar(erb_idx, env_skew.mean(1), yerr=env_skew.std(1) / np.sqrt(env_skew.shape[1]), color=condition_colors[i])
    axes[0,1].set_ylabel("Skew")
    axes[1,1].errorbar(erb_idx, env_kurt.mean(1), yerr=env_kurt.std(1) / np.sqrt(env_kurt.shape[1]), color=condition_colors[i])
    axes[1,1].set_ylabel("Kurtosis")
    axes[1,0].set_xticks(erb_pos, labels=erb_labels)

plt.tight_layout()
fig.text(0.5, 0.99, "Envelope Moments", ha="center")
fig.text(0.5, 0.00, "Cochlear Channel (Hz)", ha="center")

In [ ]:
fig.savefig("../figures/cr_song_moments.pdf")

In [ ]:
pn_mps = collect_stats(noise_stats, "modulation_power", axis=2)[1:-1].squeeze()
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(1, 2.1), dpi=150, subplot_kw={'xticks': [], 'yticks': []})
for statfile, ax in zip((colony_stats, song_stats), axes):
    envelope_mps = collect_stats(statfile, "modulation_power", axis=2).mean(2)[1:-1]
    env_mps_rel = 10 * np.log10(envelope_mps / pn_mps)
    pos = ax.imshow(env_mps_rel, vmin=-20, vmax=20, aspect="auto", cmap="seismic")
    # pos = ax.imshow(envelope_mps, aspect="auto")
axes[1].set_yticks(erb_pos, labels=erb_labels)
axes[1].set_xticks(mps_pos, labels=mps_labels)
axes[1].set_ylabel("Cochlear Channel (Hz)")
axes[0].set_title("Colony", fontsize=6)
axes[1].set_title("Song", fontsize=6)
fig.text(0.5, 0.00, "Modulation Channel (Hz)", ha="center")

cax = fig.add_axes((0.95, 0.11, 0.02, 0.35))
cax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
cax.yaxis.tick_right()
cbar = fig.colorbar(pos, cax=cax)

In [ ]:
fig.savefig("../figures/cr_song_mps.pdf")

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(3, 1.5), dpi=150)
for statfile, ax in zip((colony_stats, song_stats), axes):
    envelope_corr = collect_stats(statfile, "envelope_corr", axis=2).mean(2)[1:-1,1:-1]
    pos = ax.imshow(envelope_corr, vmin=-1, vmax=1, cmap="seismic")
axes[0].set_yticks(erb_pos, labels=erb_labels)
axes[0].set_xticks(erb_pos, labels=erb_labels)
axes[0].set_ylabel("Cochlear Channel (Hz)")
axes[0].set_xlabel("Cochlear Channel (Hz)")
axes[0].set_title("Colony", fontsize=6)
axes[1].set_title("Song", fontsize=6)

cax = fig.add_axes((0.95, 0.11, 0.01, 0.76))
cax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
cax.yaxis.tick_right()
cbar = fig.colorbar(pos, cax=cax)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(3.4, 2.2), dpi=150, 
                         subplot_kw={'xticks': [], 'yticks': []})
for j, statfile in enumerate((colony_stats, song_stats)):
    env_c1 = collect_stats(statfile, "envelope_c1_correlations", axis=3).mean(3)[1:-1,1:-1]
    for k, chan in enumerate((1,2,3)):
        channel = c12_filter_locations[chan + 1]
        pos = axes[j, k].imshow(env_c1[:, :, chan], vmin=-1, vmax=1, cmap="seismic")
        if j == 0:
            axes[j, k].set_title(f"{channel:.1f} Hz", fontsize=6)
    
# axes[0,0].set_yticks(erb_pos, labels=erb_labels)
axes[0,0].set_ylabel("Colony")
axes[1,0].set_ylabel("Song")
axes[1,1].set_xticks(erb_pos, labels=erb_labels)
# axes[0,0].set_ylabel("Cochlear Channel (Hz)")
fig.text(0.5, 0.00, "Cochlear Channel (Hz)", ha="center")

#plt.tight_layout()
cax = fig.add_axes((0.94, 0.1, 0.015, 0.35))
cax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
cax.yaxis.tick_right()
cbar = fig.colorbar(pos, cax=cax)

In [ ]:
fig.savefig("../figures/cr_song_c1_corr.pdf")